In [ ]:
!pip install virtualenv
# Create a new virtual environment
!virtualenv env_first_senti
# Activate the virtual environment
!source env_first_senti/bin/activate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 25.4 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 386ms
  creator CPython3Posix(dest=/content/env_first_senti, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.3.1, setuptools==69.0.2, wheel==0.42.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset
import numpy as np
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Load the CSV file
file_path = '/content/drive/MyDrive/updated_final_merged_data.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())

                      ts                                              title  \
0  20070101 07:12 PM EST   Apple posts options expenses, stands by CEO Jobs   
1  20070101 10:03 AM EST  Seoul antitrust body forms team on Qualcomm-re...   
2  20070101 08:39 AM EST            AT&T; closes $86 billion BellSouth deal   
3  20070101 08:39 AM EST   Apple posts options expenses, stands by CEO Jobs   
4  20070101 07:17 PM EST                    Washington pays tribute to Ford   

  ts_standardized           entities  
0   1/2/2007 0:12  [('AAPL', 'ORG')]  
1  1/1/2007 15:03  [('QCOM', 'ORG')]  
2  1/1/2007 13:39     [('T', 'ORG')]  
3  1/1/2007 13:39  [('AAPL', 'ORG')]  
4   1/2/2007 0:17     [('F', 'ORG')]  


In [ ]:
# Function to load the trained model
def load_model(model_path, num_labels, device):
    model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=num_labels)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

In [ ]:
# Function to tokenize headlines
def tokenize_headlines(tokenizer, headlines):
    return tokenizer(headlines, padding=True, truncation=True, return_tensors="pt")


In [ ]:
def make_predictions_with_confidence(model, tokenized_texts, device):
    tokenized_texts = {key: val.to(device) for key, val in tokenized_texts.items()}
    with torch.no_grad():
        outputs = model(**tokenized_texts)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
        predictions = torch.argmax(probabilities, dim=1)
        confidences = probabilities.max(dim=1).values
    return predictions.cpu().numpy(), confidences.cpu().numpy()

In [ ]:
# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# Tokenize the headlines
df['tokenized_titles'] = df['title'].apply(lambda x: tokenize_headlines(tokenizer, x))

In [ ]:
# Load your trained model
model_path = '/content/drive/MyDrive/Finetuned_RoBERTa.pth'  # Update this path
model = load_model(model_path, num_labels=3, device=device)  # Update num_labels if different

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Make predictions with confidence
df['predictions'], df['confidences'] = zip(*df.apply(lambda row: make_predictions_with_confidence(model, row['tokenized_titles'], device), axis=1))

In [ ]:
# Display the DataFrame with predictions and confidences
print(df[['title', 'predictions', 'confidences']].head())

                                               title predictions   confidences
0   Apple posts options expenses, stands by CEO Jobs         [1]   [0.9989818]
1  Seoul antitrust body forms team on Qualcomm-re...         [1]   [0.9988551]
2            AT&T; closes $86 billion BellSouth deal         [2]  [0.99048644]
3   Apple posts options expenses, stands by CEO Jobs         [1]   [0.9989818]
4                    Washington pays tribute to Ford         [1]   [0.9985335]


In [ ]:
df.head(25)

,ts,title,ts_standardized,entities,tokenized_titles,predictions,confidences
0,20070101 07:12 PM EST,"Apple posts options expenses, stands by CEO Jobs",1/2/2007 0:12,"[('AAPL', 'ORG')]","[input_ids, attention_mask]",[1],[0.9989818]
1,20070101 10:03 AM EST,Seoul antitrust body forms team on Qualcomm-re...,1/1/2007 15:03,"[('QCOM', 'ORG')]","[input_ids, attention_mask]",[1],[0.9988551]
2,20070101 08:39 AM EST,AT&T; closes $86 billion BellSouth deal,1/1/2007 13:39,"[('T', 'ORG')]","[input_ids, attention_mask]",[2],[0.99048644]
3,20070101 08:39 AM EST,"Apple posts options expenses, stands by CEO Jobs",1/1/2007 13:39,"[('AAPL', 'ORG')]","[input_ids, attention_mask]",[1],[0.9989818]
4,20070101 07:17 PM EST,Washington pays tribute to Ford,1/2/2007 0:17,"[('F', 'ORG')]","[input_ids, attention_mask]",[1],[0.9985335]
5,20070102 11:53 PM EST,Appleby right at home in bid for fourth Kapalu...,1/3/2007 4:53,"[('AAPL', 'ORG')]","[input_ids, attention_mask]",[2],[0.9882812]
6,20070102 08:18 PM EST,Apple options probe spotlights ex-officials: p...,1/3/2007 1:18,"[('AAPL', 'ORG')]","[input_ids, attention_mask]",[1],[0.9989188]
7,20070102 06:55 PM EST,"Make way for ""Emergency"" on ABC",1/2/2007 23:55,"[('ABC', 'ORG')]","[input_ids, attention_mask]",[1],[0.9990695]
8,20070102 06:55 PM EST,"ABC has ""Smith"" pilot in crosshairs",1/2/2007 23:55,"[('ABC', 'ORG')]","[input_ids, attention_mask]",[1],[0.9980829]
9,20070102 06:06 PM EST,Toyota seen outpacing Ford in U.S. December sales,1/2/2007 23:06,"[('F', 'ORG')]","[input_ids, attention_mask]",[2],[0.9961343]


In [ ]:
# Function to categorize confidence scores
def categorize_confidence(confidence):
    if confidence < 0.30:
        return 'Poor'
    elif 0.30 <= confidence < 0.70:
        return 'Average'
    else:
        return 'Excellent'

# Apply the function to the 'confidences' column
# Assuming confidences are wrapped in a list, we extract the first element
df['confidence_category'] = df['confidences'].apply(lambda x: categorize_confidence(x[0]))

# Display the DataFrame with the new column
print(df[['title', 'predictions', 'confidences', 'confidence_category']])

                                                    title predictions  \
0        Apple posts options expenses, stands by CEO Jobs         [1]   
1       Seoul antitrust body forms team on Qualcomm-re...         [1]   
2                 AT&T; closes $86 billion BellSouth deal         [2]   
3        Apple posts options expenses, stands by CEO Jobs         [1]   
4                         Washington pays tribute to Ford         [1]   
...                                                   ...         ...   
469180  BRIEF-PepsiCo announced as Twitter launch part...         [1]   
469181  RPT-Fear of failure rises as MAR tears into bo...         [0]   
469182  Tesla removes 'autopilot' from China website a...         [1]   
469183  Israel Discount Bank Q2 profit boosted by one-...         [2]   
469184  BRIEF-Workday on Monday expects to announce a ...         [2]   

         confidences confidence_category  
0        [0.9989818]           Excellent  
1        [0.9988551]           Excell

In [ ]:
df.head(25)

,ts,title,ts_standardized,entities,tokenized_titles,predictions,confidences,confidence_category
0,20070101 07:12 PM EST,"Apple posts options expenses, stands by CEO Jobs",1/2/2007 0:12,"[('AAPL', 'ORG')]","[input_ids, attention_mask]",[1],[0.9989818],Excellent
1,20070101 10:03 AM EST,Seoul antitrust body forms team on Qualcomm-re...,1/1/2007 15:03,"[('QCOM', 'ORG')]","[input_ids, attention_mask]",[1],[0.9988551],Excellent
2,20070101 08:39 AM EST,AT&T; closes $86 billion BellSouth deal,1/1/2007 13:39,"[('T', 'ORG')]","[input_ids, attention_mask]",[2],[0.99048644],Excellent
3,20070101 08:39 AM EST,"Apple posts options expenses, stands by CEO Jobs",1/1/2007 13:39,"[('AAPL', 'ORG')]","[input_ids, attention_mask]",[1],[0.9989818],Excellent
4,20070101 07:17 PM EST,Washington pays tribute to Ford,1/2/2007 0:17,"[('F', 'ORG')]","[input_ids, attention_mask]",[1],[0.9985335],Excellent
5,20070102 11:53 PM EST,Appleby right at home in bid for fourth Kapalu...,1/3/2007 4:53,"[('AAPL', 'ORG')]","[input_ids, attention_mask]",[2],[0.9882812],Excellent
6,20070102 08:18 PM EST,Apple options probe spotlights ex-officials: p...,1/3/2007 1:18,"[('AAPL', 'ORG')]","[input_ids, attention_mask]",[1],[0.9989188],Excellent
7,20070102 06:55 PM EST,"Make way for ""Emergency"" on ABC",1/2/2007 23:55,"[('ABC', 'ORG')]","[input_ids, attention_mask]",[1],[0.9990695],Excellent
8,20070102 06:55 PM EST,"ABC has ""Smith"" pilot in crosshairs",1/2/2007 23:55,"[('ABC', 'ORG')]","[input_ids, attention_mask]",[1],[0.9980829],Excellent
9,20070102 06:06 PM EST,Toyota seen outpacing Ford in U.S. December sales,1/2/2007 23:06,"[('F', 'ORG')]","[input_ids, attention_mask]",[2],[0.9961343],Excellent


In [ ]:

df.to_csv('sentiment_final_1.csv', index=False)
from google.colab import files
files.download('sentiment_final_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>